# Using Commercial LLMs with LangChain

Here we will see briefly how you can use popular commercial LLM APIs with LangChain including

- OpenAI GPT (Paid)
- Google Gemini (Paid and Free)

## Install Dependencies

In [ ]:
# ============================================================================
# REQUIRED DEPENDENCIES
# ============================================================================
# Uncomment and run these commands to install the necessary packages:
#
# langchain: Core LangChain library providing chains, prompts, and abstractions
# langchain-openai: Integration package for OpenAI's GPT models
# langchain-google-genai: Integration package for Google's Gemini models
#
# The -qq flag suppresses output for cleaner installation
# ============================================================================

# !pip install -qq langchain==0.3.11
# !pip install -qq langchain-openai==0.2.12
# !pip install -qq langchain-google-genai==2.0.6

## Load OpenAI API Credentials


In [ ]:
# ============================================================================
# METHOD 1: Interactive API Key Input (Alternative Approach)
# ============================================================================
# Use getpass for secure input - the key won't be visible while typing
# This is useful for notebooks shared publicly where you don't want to
# expose your API key in the code
# ============================================================================

# from getpass import getpass
# openai_key = getpass("Enter your OpenAI API Key: ")

In [1]:
# ============================================================================
# METHOD 2: Load API Keys from Environment File (Recommended Approach)
# ============================================================================
# Using python-dotenv to load API keys from a .env file is the best practice:
# 1. Create a .env file in your project root with:
#    OPENAI_API_KEY=your_openai_key_here
#    GOOGLE_API_KEY=your_google_key_here
# 2. Add .env to your .gitignore to prevent accidental commits
# 3. LangChain automatically reads OPENAI_API_KEY and GOOGLE_API_KEY from env
# ============================================================================

from dotenv import load_dotenv
import os

# Load environment variables from .env file
# Returns True if .env file was found and loaded successfully
load_dotenv()

True

## Use OpenAI ChatGPT with LangChain

In [9]:
# ============================================================================
# BASIC LANGCHAIN CHAIN WITH OPENAI GPT
# ============================================================================
# This demonstrates the fundamental pattern of LangChain:
# 1. Create a prompt template with placeholders (e.g., {topic})
# 2. Initialize an LLM (ChatOpenAI in this case)
# 3. Chain them together using the pipe (|) operator (LCEL syntax)
# 4. Invoke the chain with input variables
# ============================================================================

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model
# - model_name: Specifies which GPT model to use (gpt-4o-mini is cost-effective)
# - temperature: Controls randomness (0 = deterministic, 1 = creative)
chatgpt = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0  # Use 0 for consistent, reproducible outputs
)

# Create a prompt template with a placeholder variable {topic}
# ChatPromptTemplate is designed for chat models (message-based)
prompt = "Explain {topic} in 2 bullet points"
prompt_template = ChatPromptTemplate.from_template(prompt)

# Create a chain using LCEL (LangChain Expression Language)
# The | operator pipes the output of one component to the next:
# prompt_template -> formats the prompt -> chatgpt -> generates response
chain = (
    prompt_template 
    | 
    chatgpt
)

# Invoke the chain with actual values for the placeholders
# Returns an AIMessage object containing the response
response = chain.invoke({"topic": "AI"})

# Access the text content from the AIMessage object
print(response.content)


- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles (navigation and decision-making), and customer service (chatbots and virtual assistants).


In [10]:
# ============================================================================
# EXAMINING THE FULL RESPONSE OBJECT
# ============================================================================
# The response is an AIMessage object that contains:
# - content: The actual text response from the model
# - additional_kwargs: Extra data like refusal info
# - response_metadata: Token usage, model info, finish reason, etc.
# - id: Unique identifier for this response
# - usage_metadata: Detailed token counts (input, output, total)
# ============================================================================

response

AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.\n\n- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles (navigation and decision-making), and customer service (chatbots and virtual assistants).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 14, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'c

## Some of the alternative method to write the above code

### Alternative 1: Using | with StrOutputParser (get string directly)


In [3]:
# ============================================================================
# ALTERNATIVE 1: Using StrOutputParser for Direct String Output
# ============================================================================
# StrOutputParser extracts the text content from AIMessage automatically
# This is useful when you only need the text and want cleaner code
# Chain flow: prompt -> LLM -> AIMessage -> StrOutputParser -> string
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt_template = ChatPromptTemplate.from_template("Explain {topic} in 2 bullet points")

# Adding StrOutputParser() to the chain extracts just the string content
# No need to access .content on the response - it's already a string!
chain = prompt_template | chatgpt | StrOutputParser()
response = chain.invoke({"topic": "AI"})
print(response)  # Direct string output - no .content needed!

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles (navigation and decision-making), and customer service (chatbots and virtual assistants).


### Alternative 2: Using RunnableSequence explicitly

In [4]:
# ============================================================================
# ALTERNATIVE 2: Using RunnableSequence Explicitly
# ============================================================================
# RunnableSequence is what the | operator creates under the hood
# Using it explicitly can be useful for:
# - Dynamic chain construction at runtime
# - Programmatically building chains from a list of components
# - When you need more control over the chain structure
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt_template = ChatPromptTemplate.from_template("Explain {topic} in 2 bullet points")

# RunnableSequence takes components as positional arguments
# This is equivalent to: prompt_template | chatgpt
chain = RunnableSequence(prompt_template, chatgpt)
response = chain.invoke({"topic": "AI"})
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.


### Alternative 3: One-liner (no intermediate variables)

In [5]:
# ============================================================================
# ALTERNATIVE 3: One-liner (Compact Style)
# ============================================================================
# For simple use cases, you can create and invoke a chain in one expression
# Pros: Concise, good for quick experiments or simple scripts
# Cons: Harder to debug, can't reuse the chain, less readable
# Best for: Quick prototypes, simple one-off queries
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Everything in a single expression - create chain and invoke immediately
response = (
    ChatPromptTemplate.from_template("Explain {topic} in 2 bullet points") 
    | ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
).invoke({"topic": "AI"})

print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.


### Alternative 4: Using pipe() method

In [6]:
# ============================================================================
# ALTERNATIVE 4: Using pipe() Method
# ============================================================================
# The pipe() method is an alternative to the | operator
# It's the method that | calls under the hood
# Useful when you want to chain programmatically or avoid operator syntax
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt_template = ChatPromptTemplate.from_template("Explain {topic} in 2 bullet points")

# prompt_template.pipe(chatgpt) is equivalent to prompt_template | chatgpt
# You can chain multiple: prompt.pipe(llm).pipe(parser)
chain = prompt_template.pipe(chatgpt)
response = chain.invoke({"topic": "AI"})
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles (navigation and decision-making), and customer service (chatbots and virtual assistants).


### Alternative 5: Direct invocation without chain

In [7]:
# ============================================================================
# ALTERNATIVE 5: Direct Invocation Without Chain
# ============================================================================
# Instead of creating a chain, you can invoke each component separately
# This gives you more control and visibility into intermediate results
# Useful for debugging or when you need to inspect/modify data between steps
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt_template = ChatPromptTemplate.from_template("Explain {topic} in 2 bullet points")

# Step 1: Format the prompt with variables -> produces ChatPromptValue (messages)
messages = prompt_template.invoke({"topic": "AI"})
# You can inspect 'messages' here if needed: print(messages.to_messages())

# Step 2: Send messages to the LLM -> produces AIMessage
response = chatgpt.invoke(messages)
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.


### Using from_messages

In [8]:
# ============================================================================
# USING from_messages FOR MULTI-ROLE PROMPTS
# ============================================================================
# from_messages allows you to specify different roles in the conversation:
# - "system": Sets the AI's behavior, personality, or context
# - "human"/"user": Represents user messages
# - "ai"/"assistant": Represents AI responses (for few-shot examples)
# 
# This is essential for production applications where you need:
# - System prompts to control behavior
# - Few-shot learning examples
# - Multi-turn conversation context
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Using from_messages with role tuples (role, content)
# The system message shapes how the AI responds
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant that explains concepts clearly."),
    ("human", "Explain {topic} in 2 bullet points")
])

chain = prompt_template | chatgpt
response = chain.invoke({"topic": "AI"})
print(response.content)

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think, learn, and make decisions like humans. This includes capabilities such as problem-solving, understanding natural language, and recognizing patterns.

- **Applications**: AI is used in various fields, including healthcare (for diagnostics), finance (for fraud detection), autonomous vehicles (for navigation), and customer service (through chatbots), enhancing efficiency and enabling new solutions to complex problems.


In [ ]:
# Examine the full AIMessage object to understand its structure
response

AIMessage(content='- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.\n\n- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 14, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_rea

## `ChatPromptTemplate.from_template` vs `ChatPromptTemplate.from_messages`

### Key Differences

| Aspect | `from_template` | `from_messages` |
|--------|-----------------|-----------------|
| **Input** | Single string | List of message tuples |
| **Roles** | Defaults to `HumanMessage` | Explicit control over roles |
| **System prompt** | Not directly supported | ✅ Supported |
| **Multi-turn context** | ❌ | ✅ Can include AI responses |
| **Use case** | Simple, single prompts | Complex chat interactions |

### Role Shortcuts for `from_messages`

| Tuple Role | Message Class |
|------------|---------------|
| `"system"` | `SystemMessage` |
| `"human"` / `"user"` | `HumanMessage` |
| `"ai"` / `"assistant"` | `AIMessage` |

### Under the Hood

```python
# from_template:
prompt = ChatPromptTemplate.from_template("Explain {topic}")

# Is equivalent to:
prompt = ChatPromptTemplate.from_messages([
    ("human", "Explain {topic}")
])
```

### When to Use

- **`from_template`**: Quick, simple single-turn prompts
- **`from_messages`**: When you need system prompts, few-shot examples, or multi-turn context

## use PromptTemplate iso ChatPromptTemplate

In [ ]:
# ============================================================================
# USING PromptTemplate WITH CHAT MODELS
# ============================================================================
# PromptTemplate (without "Chat") was originally designed for completion models
# When used with ChatOpenAI, it works but gets auto-converted to HumanMessage
# 
# Key difference:
# - PromptTemplate.invoke() -> StringPromptValue (plain string)
# - ChatPromptTemplate.invoke() -> ChatPromptValue (list of messages)
#
# LangChain handles the conversion automatically, but it's cleaner to use
# ChatPromptTemplate with chat models for semantic correctness
# ============================================================================

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize Chat Model
chatgpt = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

# PromptTemplate produces a string (not messages)
# LangChain auto-converts it to a HumanMessage when sent to ChatOpenAI
prompt_template = PromptTemplate.from_template("Explain {topic} in 2 bullet points")

# Create and run chain - works the same way
chain = prompt_template | chatgpt
response1 = chain.invoke({"topic": "AI"})
print(response1.content)  # Fixed: using response1 instead of response

- **Definition**: Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, particularly computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and understanding natural language.

- **Applications**: AI is utilized across various fields, including healthcare (diagnosis and treatment recommendations), finance (fraud detection and algorithmic trading), autonomous vehicles, customer service (chatbots), and many more, enhancing efficiency and decision-making.


In [ ]:
# Examine response1 - notice the structure is the same as using ChatPromptTemplate
# This shows that LangChain's automatic conversion works seamlessly
response1

AIMessage(content='- **Definition and Functionality**: Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think, learn, and perform tasks autonomously, often using algorithms and data to improve their performance over time.\n\n- **Applications and Impact**: AI is utilized across various sectors, including healthcare, finance, and transportation, enhancing efficiency, decision-making, and personalization, while also raising ethical considerations regarding privacy, bias, and job displacement.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 14, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', '

## `PromptTemplate` vs `ChatPromptTemplate`

### Key Differences

| Aspect | `PromptTemplate` | `ChatPromptTemplate` |
|--------|------------------|----------------------|
| **Output type** | `StringPromptValue` (string) | `ChatPromptValue` (list of messages) |
| **Designed for** | Completion models (text-in, text-out) | Chat models (messages-in, message-out) |
| **System prompt** | ❌ Not supported | ✅ Supported |
| **Multiple roles** | ❌ Single string only | ✅ system, human, ai |
| **With ChatOpenAI** | Auto-converts to HumanMessage | Native message format |

### Under the Hood

```python
# PromptTemplate produces a string:
prompt = PromptTemplate.from_template("Explain {topic}")
result = prompt.invoke({"topic": "AI"})
# Output: StringPromptValue -> "Explain AI"

# ChatPromptTemplate produces messages:
chat_prompt = ChatPromptTemplate.from_template("Explain {topic}")
result = chat_prompt.invoke({"topic": "AI"})
# Output: ChatPromptValue -> [HumanMessage(content='Explain AI')]
```

### When to Use Each

| Use Case | Recommendation |
|----------|----------------|
| Simple prompts with ChatOpenAI | Either works ✅ |
| Need system prompt | `ChatPromptTemplate` |
| Need multi-turn/few-shot | `ChatPromptTemplate` |
| Working with completion models (legacy) | `PromptTemplate` |
| Building complex chat applications | `ChatPromptTemplate` |

### Recommendation

For **chat models** like ChatOpenAI, prefer `ChatPromptTemplate` because:
- ✅ Semantically correct (designed for chat models)
- ✅ Supports system prompts and multiple roles
- ✅ More explicit about the message structure
- ✅ Future-proof for complex use cases

In [11]:
# ============================================================================
# DEMONSTRATION: PromptTemplate vs ChatPromptTemplate Output Types
# ============================================================================
# This cell shows the actual difference in what each template produces
# when you invoke them with variables
# ============================================================================

# PromptTemplate produces a StringPromptValue (essentially a string wrapper)
prompt = PromptTemplate.from_template("Explain {topic}")
result = prompt.invoke({"topic": "AI"})
print("=== PromptTemplate Output ===")
print(f"Type: {type(result)}")  # StringPromptValue
print(f"String: {result.to_string()}")  # "Explain AI"

print()  # Blank line for readability

# ChatPromptTemplate produces a ChatPromptValue (list of messages)
chat_prompt = ChatPromptTemplate.from_template("Explain {topic}")
result = chat_prompt.invoke({"topic": "AI"})
print("=== ChatPromptTemplate Output ===")
print(f"Type: {type(result)}")  # ChatPromptValue
print(f"Messages: {result.to_messages()}")  # [HumanMessage(content='Explain AI')]

<class 'langchain_core.prompt_values.StringPromptValue'>
Explain AI
<class 'langchain_core.prompt_values.ChatPromptValue'>
[HumanMessage(content='Explain AI', additional_kwargs={}, response_metadata={})]


## Load Gemini API credentials

Run this section only if you are using Google Gemini

In [12]:
# ============================================================================
# LOAD GOOGLE GEMINI API CREDENTIALS
# ============================================================================
# For Google Gemini, you need GOOGLE_API_KEY in your .env file
# Get your API key from: https://makersuite.google.com/app/apikey
# Gemini offers a generous free tier, making it great for learning!
# ============================================================================

from dotenv import load_dotenv
import os

# Load environment variables (including GOOGLE_API_KEY)
load_dotenv()

True

In [17]:
# ============================================================================
# USING GOOGLE GEMINI WITH LANGCHAIN
# ============================================================================
# LangChain provides a unified interface - the code structure is identical
# to OpenAI! Only the model class changes:
# - OpenAI: ChatOpenAI from langchain_openai
# - Gemini: ChatGoogleGenerativeAI from langchain_google_genai
#
# This abstraction is one of LangChain's key benefits - easily swap models!
# ============================================================================

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Gemini model
# Available models: gemini-pro, gemini-1.5-pro, gemini-1.5-flash, gemini-2.5-flash
# Flash models are faster and cheaper, Pro models are more capable
gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Same prompt template syntax works across all LLMs
PROMPT = "Explain {topic} in 2 bullets"
prompt = ChatPromptTemplate.from_template(PROMPT)

# Same chain pattern - this is the power of LangChain's abstraction!
chain = (
    prompt
    |
    gemini
)

# Invoke works exactly the same way
response = chain.invoke({"topic": "AI"})
print(response.content)

Here are two bullets explaining AI:

*   **AI (Artificial Intelligence) refers to computer systems designed to perform tasks that typically require human intelligence.** This includes abilities like understanding language, recognizing images, making decisions, and solving problems.
*   **These systems learn from data to identify patterns, make predictions, and adapt their behavior without explicit programming for every scenario.** This allows them to automate complex cognitive tasks and continuously improve their performance.


## Summary & Key Takeaways

### 🎯 Core Concepts Covered

1. **LangChain's Value Proposition**: Provides a unified interface to work with different LLM providers (OpenAI, Google, etc.) using the same code patterns.

2. **LCEL (LangChain Expression Language)**: The `|` pipe operator chains components together:
   ```python
   chain = prompt_template | llm | output_parser
   ```

3. **Prompt Templates**:
   - `ChatPromptTemplate.from_template()` - Simple single-turn prompts
   - `ChatPromptTemplate.from_messages()` - Multi-role prompts with system messages
   - `PromptTemplate` - Legacy, works but prefer ChatPromptTemplate for chat models

4. **Chain Invocation**: Always use `.invoke({"key": "value"})` with a dictionary of variables

### 🔑 Best Practices

- Store API keys in `.env` files, never hardcode them
- Use `temperature=0` for consistent, reproducible outputs
- Prefer `ChatPromptTemplate` over `PromptTemplate` for chat models
- Use `StrOutputParser` when you only need the text response
- System prompts (via `from_messages`) help control AI behavior

### 📚 Next Steps

- Explore output parsers for structured data (JSON, Pydantic models)
- Learn about memory for conversation history
- Dive into RAG (Retrieval Augmented Generation)
- Build agents with tools
